<a href="https://colab.research.google.com/github/MonikaRudra/FDPLLM/blob/main/SPAM_classifier_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("spam.csv",encoding="ISO-8859-1")
df = df.rename(columns={'v1':'target','v2':'email'})
df = df[["email","target"]]

In [ ]:
# data cleaning
import re
def clean(str_):
  str_ = str_.lower()
  str_ = re.sub('[^A-Za-z0-9]+'," ",str_)
  return str_

In [ ]:
df["cleaned_email"] = df["email"].apply(clean)

<ipython-input-4-7054d8fd27cb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cleaned_email"] = df["email"].apply(clean)


In [ ]:
sentences = df['cleaned_email'].tolist()

In [ ]:
sent_tokens = []
for sent in sentences:
  sent_tokens.append(sent.split(" "))

In [ ]:
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec(sent_tokens,min_count=1,vector_size=100)

In [ ]:
import numpy as np
sent_vec = []
for sent in sentences:
  temp_vec = []
  for word in sent.split(" "):
    temp_vec.append(model.wv[word])
  sent_vec.append(np.array(temp_vec).mean(axis=0))

In [ ]:
df['feature'] = sent_vec

In [ ]:
df.head(3)

,email,target,cleaned_email,feature
0,"Go until jurong point, crazy.. Available only ...",ham,go until jurong point crazy available only in ...,"[-0.25417227, 0.49219468, -0.0029789405, 0.072..."
1,Ok lar... Joking wif u oni...,ham,ok lar joking wif u oni,"[-0.26738754, 0.52055097, -0.0027107527, 0.071..."
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,free entry in 2 a wkly comp to win fa cup fina...,"[-0.26914394, 0.54020774, -0.006450683, 0.0676..."


In [ ]:
X = np.array(sent_vec)
Y = df['target']

In [ ]:
X.shape

(5572, 100)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test, y_train,y_test = train_test_split(X,Y, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier()

In [ ]:
clf.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test,y_pred)

0.9641255605381166

In [ ]:
unknown_email ="free free hey apply for credit card and loan on your bank account"

In [ ]:
temp_vec
for word in unknown_email.split(" "):
  temp_vec.append(model.wv[word])
clf.predict([np.array(temp_vec).mean(axis=0)])

array(['spam'], dtype=object)